# FIFA API: Schedule

#### Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_stiles as altstiles

In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('stiles')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

---

In [5]:
results_dicts = pd.read_json(
    "https://api.fifa.com/api/v3/calendar/matches?language=en&count=500&idSeason=255711"
)["Results"]

In [6]:
match_dict = {key: value for key, value in results_dicts.items() if len(value) > 1}

In [7]:
src = pd.DataFrame(match_dict).T

In [8]:
src.columns = src.columns.str.lower()

In [9]:
src[["locale", "group_name"]] = pd.json_normalize(src["groupname"])[0].apply(
    pd.Series, dtype="object"
)

In [10]:
src["home_team"] = pd.json_normalize(src["home"])["Abbreviation"]
src["away_team"] = pd.json_normalize(src["away"])["Abbreviation"]

In [11]:
src[["locale", "stage_name"]] = pd.json_normalize(src["stagename"])[0].apply(pd.Series)

In [12]:
src_slim = src[
    ["idmatch", "date", "matchnumber", "group_name", "home_team", "away_team"]
].copy()

In [13]:
src_slim["local_date"] = pd.to_datetime(src_slim["date"]).dt.strftime("%Y-%m-%d")
src_slim["local_time"] = pd.to_datetime(src_slim["date"]).dt.strftime("%H:%M:%S")

In [14]:
src_slim["pst_date"] = (
    pd.to_datetime(src_slim["date"]).dt.tz_convert("US/Pacific").dt.strftime("%Y-%m-%d")
)

In [15]:
src_slim["pst_time"] = (
    pd.to_datetime(src_slim["date"]).dt.tz_convert("US/Pacific").dt.strftime("%H:%M:%S")
)

In [16]:
src_slim.drop("date", axis=1, inplace=True)

In [17]:
df = src_slim.copy()

----

#### USA

In [18]:
df.query('home_team == "USA" or away_team == "USA"')

,idmatch,matchnumber,group_name,home_team,away_team,local_date,local_time,pst_date,pst_time
3,400235455,4,Group B,USA,WAL,2022-11-21,19:00:00,2022-11-21,11:00:00
19,400235457,20,Group B,ENG,USA,2022-11-25,19:00:00,2022-11-25,11:00:00
35,400235456,34,Group B,IRN,USA,2022-11-29,19:00:00,2022-11-29,11:00:00


In [19]:
dates = pd.DataFrame(df.local_date.astype(str).unique(), columns=["match_date"])

In [20]:
dates["match_date"].to_list()

['2022-11-20',
 '2022-11-21',
 '2022-11-22',
 '2022-11-23',
 '2022-11-24',
 '2022-11-25',
 '2022-11-26',
 '2022-11-27',
 '2022-11-28',
 '2022-11-29',
 '2022-11-30',
 '2022-12-01',
 '2022-12-02',
 '2022-12-03',
 '2022-12-04',
 '2022-12-05',
 '2022-12-06',
 '2022-12-09',
 '2022-12-10',
 '2022-12-13',
 '2022-12-14',
 '2022-12-17',
 '2022-12-18']

---

## Export

In [21]:
df.to_csv("data/processed/world_cup_matches_schedule.csv", index=False)